In [ ]:
rm(list=ls())
libraries = c("dplyr","magrittr","tidyr","reshape2","ggplot2","ggpubr","extrafont","openxlsx","readxl","RColorBrewer","MetBrewer","zoo","RVAideMemoire"
              ,"writexl","gridExtra","MASS","readr","stats","boot","pracma","stringr","mixdist","corpcor","rstan","bayesplot","graphics")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

theme_set(theme_bw())
setwd("../MPX_depletion_susceptibles")

In [ ]:
df <- read.csv("data/usmpx_combined.csv")

In [ ]:
df$Country %>% unique 
country <- df$Country %>% unique

In [ ]:
c_number <- rep(0,length(country))
for(i in 1:length(country)){
  dff <- df %>% filter(Country==country[i])
  dff$Date <- as.Date(dff$Date)
  
  dff <- dff %>% dplyr::select("Cases","Cumulative_cases")  
  row_sub = apply(dff, 1, function(row) all(row !=0 ))
  num <- grep("TRUE",row_sub)
  dff<- dff[row_sub,] 
  dff %<>% cbind(num)
  df_fin <- dff[!apply(as.data.frame(dff$Cumulative_cases), 1, function(row) all(row <=10 )),]
  
  if(is.na(df_fin$Cases[1])) {c_number[i]=FALSE} else {if(df_fin$Cases[1]) {c_number[i]=i}} 
}
c_number[c_number==0] <-NA
c_number <- c_number[!is.na(c_number)]

In [ ]:
c_number %>% length()

In [ ]:
i=2
N_pop <- c(5073187,7303398,3030646,39995077,5922618,3612314,1008350,707109,22085563,10916760,1474265,1893410,12808884,6845874,3219171,4539130,4682633,6257958,7126375,10116069,5787008,2960075,6188111,1988536,3185426,1389741,9388414,2129190,20365879,10620168,11852036,4000953,4318492,13062764,3197890,1106341,5217037,7023788,29945493,335184010,3373162,8757467,7901429,5935064)
  dff <- df %>% filter(Country==country[c_number[i]])
  dff$Date <- as.Date(dff$Date)
  
  dff <- dff %>% dplyr::select("Cases","Cumulative_cases")  
  row_sub = apply(dff, 1, function(row) all(row !=0 ))
  num <- grep("TRUE",row_sub)
  dff<- dff[row_sub,] 
  dff %<>% cbind(num)
  df_fin <- dff[!apply(as.data.frame(dff$Cumulative_cases), 1, function(row) all(row <=10 )),]
  df_fin1 <- dff                     
  
  it = df_fin$Cumulative_cases
  df_fin
  T= length(it)
  TT = df_fin$num[T]
  N_pop=N_pop[i]

In [ ]:
source("src/R/colour.R")

In [ ]:
source("src/Stan/growth_estim.stan")

In [ ]:
source("src/R/US_states_growth_estim_r.R") #excluding c_number[12]; Rhat 7,15

In [ ]:
c_number <- rep(0,length(country))
for(i in 1:length(country)){
  dff <- df %>% filter(Country==country[i])
  date_n<- as.Date(dff$Date) %>% as.numeric()
  dff %<>% cbind(date_n)
  dff <- dff %>% dplyr::select("Cases","Cumulative_cases","date_n")  
  row_sub = apply(dff, 1, function(row) all(row !=0 ))
  num <- grep("TRUE",row_sub)
  dff<- dff[row_sub,] 
  dff %<>% cbind(num)
  df_fin <- dff[!apply(as.data.frame(dff$Cumulative_cases), 1, function(row) all(row <=10 )),]
  
  if(is.na(df_fin$Cases[1])) {c_number[i]=FALSE} else {if(df_fin$Cases[1]) {c_number[i]=i}} 
}
c_number[c_number==0] <-NA
c_number <- c_number[!is.na(c_number)]
intro_date <- rep(0,length(c_number))
numb_i <- c(1:length(c_number))
numb_i <- numb_i[-c(7,11,12,16,41,45)]
for(i in numb_i){
    dff <- df %>% filter(Country==country[c_number[i]])
  date_n<- as.Date(dff$Date) %>% as.numeric()
  dff %<>% cbind(date_n)
  dff <- dff %>% dplyr::select("Cases","Cumulative_cases","date_n")  
  row_sub = apply(dff, 1, function(row) all(row !=0 ))
  num <- grep("TRUE",row_sub)
  dff<- dff[row_sub,] 
  dff %<>% cbind(num)
  df_fin <- dff[!apply(as.data.frame(dff$Cumulative_cases), 1, function(row) all(row <=10 )),]
  intro_date[i] <- df_fin$date_n[1] 
}
intro_date[intro_date==0] <- NA
intro_date <- intro_date[!is.na(intro_date)]
intro_date <- intro_date-min(intro_date)
intro_df <- cbind(country[c_number[-c(7,11,12,16,41,45)]],intro_date)
intro_date <- intro_date-min(intro_date)
intro_df <- cbind(country[c_number[-c(7,11,12,16,41,45)]],intro_date)

In [ ]:
numb_i <- c(1:length(c_number))
numb_i <- numb_i[-c(7,11,12,16,41,45)]
range_summary <- matrix(rep(0,5*(length(c_number))), nrow = length(c_number), ncol = 5) %>% as.data.frame()
for(i in numb_i){
range_df <- read.csv(paste0("output/csv/us/",country[c_number[i]],"_growth.csv"))
n_of_dates <- nrow(range_df)
range_summary[i,] <- c(country[c_number[i]],c_number[i],range_df$lower[1], range_df$upper[n_of_dates],range_df$date[1])
    }
colnames(range_summary) <- c("Country","Country_number","lower","upper","date_peak")
latest_cases <- rep(0,length(c_number)) 
for(i in numb_i){
dfff <- df %>% filter(Country==country[c_number[i]]) %>% dplyr::select(Cumulative_cases) 
latest_n <- df %>% filter(Country==country[c_number[i]]) %>% dplyr::select(Cumulative_cases) %>% nrow()
latest_cases[i] <- dfff[latest_n,]
}

pop_size <- c(5073187,7303398,3030646,39995077,5922618,3612314,1008350,707109,22085563,10916760,1474265,1893410,12808884,6845874,3219171,2934582,4539130,4682633,6257958,7126375,10116069,5787008,2960075,6188111,1988536,3185426,1389741,9388414,2129190,20365879,10620168,11852036,4000953,4318492,13062764,3197890,1106341,5217037,7023788,29945493,335184010,3373162,8757467,7901429,1782959,5935064)
msm_size <- c(40600,110344,19264,792750,73357,43313,13049,36775,340163,131374,15411,9907,199486,70103,20753,22900,47034,41492,84465,111625,113860,83027,18992,70783,13199,51726,14122,132520,17969,371087,103010,144367,37739,61607,162745,NA,23815,36316,73639,371781,4503080,33294,112785,111960,13068,59078)

latest_cases %<>%  as.data.frame() 
range_summary %<>% cbind(latest_cases, pop_size, msm_size)
colnames(range_summary) <- c("Country","Country_number","lower","upper","date_peak","latest_cases","pop_size","msm_size")
range_summary[range_summary$latest_cases==0,] <-NA
#range_summary[range_summary$Country==0,] <-NA
range_summary %<>% drop_na()

a="Both limits";b="Only lower limit"
label <-  c(b,a,b,b,b,a,a,a,a,a,b,b,a,a,a,a,b,b,b,b,b,b,b,a,b,a,a,a,b,b,a,b,a,a,a,a,a,a,b)
range_summary %<>% cbind(label)
upper1 <- ifelse(range_summary$label=="Only lower limit",0,range_summary$upper)
upper_forplot <- ifelse(range_summary$label=="Only lower limit",range_summary$upper,0)
perpop_lower <- as.numeric(range_summary$lower)/as.numeric(range_summary$pop_size)
perpop_upper <- as.numeric(range_summary$upper)/as.numeric(range_summary$pop_size)
range_summary %<>% cbind(upper1,upper_forplot,perpop_lower,perpop_upper) 


range_summary <- range_summary[!is.na(range_summary$msm_size),]
Country1 <- ifelse(range_summary$label=="Only lower limit",range_summary$Country,NA)
range_summary[range_summary==0] <-NA

permsm_lower <- as.numeric(range_summary$lower)/as.numeric(range_summary$msm_size)
permsm_upper <- as.numeric(range_summary$upper)/as.numeric(range_summary$msm_size)
range_summary %<>% cbind(permsm_lower,permsm_upper,Country1)
range_summary <- range_summary[order(permsm_lower,decreasing=TRUE),]
range_summary %<>% cbind(c("aa","ab","ac","ad","ae","af","ag","ah","ai","aj","ak","al","am","an","ao","ap","aq","ar","as","at","au","av","aw","ax","ay","az","ba","bb","bc","bd","be","bf","bg","bh","bi","bj","bk","bl","bm"))
colnames(range_summary) <- c("Country","Country_number","lower","upper","date_peak","latest_cases","pop_size","msm_size","label","upper1","upper_forplot","perpop_lower","perpop_upper","permsm_lower","permsm_upper","Country1","Country2")
Country3 <- ifelse(range_summary$label=="Only lower limit",range_summary$Country2,NA)
latest_cases1 <-  ifelse(range_summary$label=="Only lower limit",range_summary$latest_cases,NA)
range_summary %<>% cbind(Country3,latest_cases1)



intro_date <- rep(0,length(range_summary$Country_number))
numb_i <- range_summary$Country_number %>% as.numeric()
for(i in 1:length(numb_i)){
    dff <- df %>% filter(Country==country[numb_i[i]])
  date_n<- as.Date(dff$Date) %>% as.numeric()
  dff %<>% cbind(date_n)
  dff <- dff %>% dplyr::select("Cases","Cumulative_cases","date_n")  
  row_sub = apply(dff, 1, function(row) all(row !=0 ))
  num <- grep("TRUE",row_sub)
  dff<- dff[row_sub,] 
  dff %<>% cbind(num)
  df_fin <- dff[!apply(as.data.frame(dff$Cumulative_cases), 1, function(row) all(row <=10 )),]
  intro_date[i] <- df_fin$date_n[1] 
}
intro_date[intro_date==0] <- NA
intro_date <- intro_date[!is.na(intro_date)]
min_date <- min(intro_date)
intro_date <- intro_date-min(intro_date)
range_summary %<>% cbind(intro_date)
intro_date_loweronly <- ifelse(range_summary$label=="Only lower limit",range_summary$intro_date,NA)
msm_size1 <- ifelse(range_summary$label=="Only lower limit",range_summary$msm_size,NA)
pop_size1 <- ifelse(range_summary$label=="Only lower limit",range_summary$pop_size,NA)
mid_prop <- (range_summary$permsm_lower+range_summary$permsm_upper)/2
range_summary %<>% cbind(intro_date_loweronly,msm_size1,pop_size1,mid_prop)
     
numb_i <- c(1:nrow(range_summary))
date_peak_adj <- rep(0,nrow(range_summary))
n_of_vax <- rep(0,nrow(range_summary)) 
                       
for(i in 1:length(numb_i)){             
date_peak_adj[i] <- range_summary$intro_date[i]+as.numeric(range_summary$date_peak[i])-1

df_vax <- read.csv("https://raw.githubusercontent.com/hiroaki-murayama/us-mpx/main/output/jynneos.csv")
df_vax_1 <- df_vax %>% filter(Jurisdiction==country[as.numeric(range_summary$Country_number[i])]) 
df_vax_1 <- cbind(as.numeric(as.Date(df_vax_1$DateDT)),as.numeric(df_vax_1$AllocatedCNT)) %>% as.data.frame()
colnames(df_vax_1) <- c("DateDT","AllocatedCNT")
    
    
numb <- rep(0,length(df_vax_1$DateDT))
X <- df_vax_1$DateDT[1:length(df_vax_1$DateDT)]-min_date
if(any(X==date_peak_adj[i])){
numb[i] <- which(X==date_peak_adj[i])
n_of_vax[i] <- df_vax_1$AllocatedCNT[numb[i]]
} 
if(all(X!=date_peak_adj[i])){
n_of_vax[i] <- 0
}
}
prop_of_vax <- n_of_vax/range_summary$msm_size
range_summary %<>% cbind(prop_of_vax,date_peak_adj)
                   
                       
range_summary

In [ ]:
### calculate score

width_1 <- 1/100000
number_loweronly <- range_summary %>% filter(label=="Only lower limit") %>% nrow()
score <- rep(1,1/width_1)

for(i in 1:nrow(range_summary)){
width_score <- round(seq(range_summary$permsm_lower[i]/width_1,range_summary$permsm_upper[i]/width_1,1))
for(j in width_score) {score[j] = score[j] + 1}

width_score_1 <- round(seq(1+range_summary$permsm_upper[i]/width_1,1/width_1,1))
if(range_summary$label[i]=="Only lower limit"){
for(k in width_score_1) {score[k] = score[k] + 1}
} 
}


### specify the range of global estimates (threshold=20)

prop_hit <- rep(NA,1/width_1)
for(i in 1:(1/width_1 -1)){
if(score[i]>=21){
    prop_hit[i] = i*width_1
}
}

prop_hit <- prop_hit[!is.na(prop_hit)]
prop_length <- length(prop_hit)

### plot

options(repr.plot.width=12,repr.plot.height=8)
range_l <- prop_hit[1]
range_u <- prop_hit[prop_length]
df_score <- cbind(seq(1,0.03/width_1,1)*width_1,score[1:(0.03/width_1)]) %>% as.data.frame()
colnames(df_score) <- c("prop","score_value") 
df_score_plot <- df_score %>% ggplot() +
    annotate("rect", xmin = range_l*100, xmax = range_u*100, ymin = -Inf, ymax = Inf, fill = met.brewer("Hokusai2",n=2)[2], alpha = 0.15) +
    geom_line(aes(x=prop*100, y=score_value), colour=met.brewer("Troy",n=15)[6], size=1.2, alpha=2) +
    geom_hline(yintercept=21, linetype="dashed", color = "#1380A1", size =0.7) + 
    theme_bw(base_size = 22) + theme(legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
    scale_x_continuous(breaks=c(0, 0.5, 1, 1.5, 2, 2.5, 3),limits=c(0,3)) +
    scale_color_manual(values=met.brewer("Cassatt2",8)[8]) + theme(axis.text.x = element_text(angle = 0, vjust = 0.5, hjust = 0.5),panel.grid.major=element_blank(), panel.grid.minor=element_blank()) +
    labs(x="Cumulative cases per MSM population at a peak of an epidemic [%]", y = "Number of US states")
df_score_plot
ggsave(df_score_plot,file = "output/fig/US_score_plot.png", width = 12, height = 8)

In [ ]:
options(repr.plot.width=18,repr.plot.height=8)
Country_label <- range_summary$Country
range_summary %>% ggplot() + 
　annotate("rect", ymin = range_l*100, ymax = range_u*100, xmin = -Inf, xmax = Inf, fill = met.brewer("Hokusai2",n=2)[2], alpha = 0.15) +
  geom_rect(aes(xmin= Country2, xmax=Country2, ymin=permsm_lower*100, ymax=permsm_upper*100, group=label, color=label, fill=label), size=3)  + geom_segment(aes(x=Country3,xend=Country3,y=permsm_upper*100,yend=2, color=label),size=1, linetype=2) +
  theme_bw(base_size = 22) + theme(legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1), legend.position="top", legend.justification='right', legend.title = element_blank(),panel.grid.major=element_blank(), panel.grid.minor=element_blank()) + scale_color_manual(values=c(met.brewer("Troy",n=15)[5],met.brewer("Troy",n=15)[10])) + 
  scale_fill_manual(values=c(met.brewer("Troy",n=15)[5],met.brewer("Troy",n=15)[10])) + scale_y_continuous(breaks=c(0,0.5, 1, 1.5,2),limits=c(0,2)) +
  labs(x="", y = "Cumulative cases per MSM population\
at a peak of an epidemic [%]\
 ") + scale_x_discrete(labels = Country_label)
ggsave(file = "output/fig/US_incidenceprop_msm.jpeg", width = 18, height = 8)

In [ ]:
options(scipen=10000000)
options(repr.plot.width=10,repr.plot.height=8)
range_summary %>% ggplot() +
geom_point(aes(x=permsm_lower*100, y=as.numeric(msm_size)),size=4,colour=met.brewer("Austria",n=2)[2]) +
theme(legend.title = element_blank()) +
#scale_color_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) + scale_fill_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) +
scale_x_continuous(breaks = seq(0, 0.85, 0.2), limits=c(0, 0.85)) + 
scale_y_continuous(breaks = seq(0, 4500000, 1000000), limits=c(0, 4500000)) + 
theme_bw(base_size = 22) + theme(legend.title = element_blank(),legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
labs(x="Cumulative cases\ 
per MSM population at a peak of an epidemic [%]", y = "MSM population size
 ")  

ggsave(file = "output/fig/correlation_msmsize_us.jpeg", width = 10, height = 8)

In [ ]:
spearman.ci(range_summary$permsm_lower, range_summary$intro_date, nrep = 1000, conf.level = 0.95)

In [ ]:
options(scipen=10000000)
options(repr.plot.width=10,repr.plot.height=8)
range_summary %>% ggplot() +
geom_point(aes(x=permsm_lower*100, y=as.numeric(msm_size)),size=4,colour=met.brewer("Austria",n=2)[2]) +
theme(legend.title = element_blank()) +
#scale_color_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) + scale_fill_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) +
scale_x_continuous(breaks = seq(0, 0.85, 0.2), limits=c(0, 0.85)) + 
scale_y_continuous(breaks = seq(0, 5500000, 1000000), limits=c(0, 5500000)) + 
theme_bw(base_size = 22) + theme(legend.title = element_blank(),legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
labs(x="Cumulative cases\ 
per MSM population at a peak of an epidemic [%]", y = "MSM population size
 ")  

ggsave(file = "output/fig/correlation_msmsize_us.jpeg", width = 10, height = 8)

In [ ]:
spearman.ci(range_summary$permsm_lower, as.numeric(range_summary$msm_size), nrep = 1000, conf.level = 0.95)

In [ ]:
options(scipen=10000000)
options(repr.plot.width=10,repr.plot.height=8)
range_summary %>% ggplot() +
geom_point(aes(x=permsm_lower, y=prop_of_vax),size=4,colour=met.brewer("OKeeffe1",n=11)[9]) +
theme(legend.title = element_blank()) +
#scale_color_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) + scale_fill_manual(values=c(met.brewer("Monet",n=9)[2],met.brewer("Monet",n=9)[8])) +
#scale_x_continuous(breaks = seq(0, 5250, 500), limits=c(0, 5250)) + 
#scale_y_continuous(breaks = seq(0, 30000, 10000), limits=c(0, 30000)) + 
theme_bw(base_size = 22) + theme(legend.title = element_blank(),legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
labs(x="Cumulative cases\ 
per MSM population at a peak of an epidemic", y = "Allocated doses of vaccine \
per MSM population
 ")  

ggsave(file = "output/fig/correlation_vax_us.jpeg", width = 10, height = 8)

In [ ]:
spearman.ci(range_summary$permsm_lower, range_summary$prop_of_vax, nrep = 1000, conf.level = 0.95)